In [4]:
from residPCA import *

In [5]:
scExp = residPCA(count_matrix_path="/Users/shayecarver/residPCA/examples/example_data.h5ad", 
               vars_to_regress=['Batch','celltype','total_counts', 'pct_counts_mt','Age','Sex'],
               object_columns=['celltype','Sex','Batch'], 
               save_image_outputs=False, 
               BIC=True, 
               vargenes_Stand_resid=3000,
               n_PCs = 150,
               random_seed=7, 
               vargenes_IterPCA=3000, 
               #path_to_directory = "./", 
               #basename=f'Morab_RNA_3kvargenes_seurat',
               variable_genes_flavor="seurat",  
               global_ct_cutoff=0.2)  
scExp.Normalize() 
scExp.Standardize()
scExp.StandardPCA_fit()
scExp.residPCA_fit()
scExp.Iter_PCA_fit()
# identify global versus cell type specific states
scExp.ID_Global_CellType_States()


NameError: name 'residPCA' is not defined